<a href="https://colab.research.google.com/github/pratik9153/Spark-ML/blob/main/Spark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=14edec0efb2aff8324175f2af497166298e1ef415049444afe2e608619aa8d6d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## Linear Regression

In [57]:
import pyspark
from pyspark.sql import SparkSession

In [58]:
spark = SparkSession.builder.appName("Crew_Requirements").getOrCreate()

In [59]:
df = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [ ]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [ ]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line",outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
only showing top 5 rows



In [ ]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [ ]:
indexed.select('Cruise_line','cruise_cat').distinct().show()

+-----------------+----------+
|      Cruise_line|cruise_cat|
+-----------------+----------+
|            Costa|       5.0|
|        Norwegian|       4.0|
|              MSC|       7.0|
|           Orient|      19.0|
|Regent_Seven_Seas|      10.0|
|           Disney|      18.0|
|         Windstar|      15.0|
|              P&O|       8.0|
|  Royal_Caribbean|       0.0|
|         Seabourn|      14.0|
|             Star|       9.0|
|         Princess|       2.0|
|          Oceania|      13.0|
|          Azamara|      16.0|
| Holland_American|       3.0|
|           Cunard|      12.0|
|        Celebrity|       6.0|
|        Silversea|      11.0|
|          Crystal|      17.0|
|         Carnival|       1.0|
+-----------------+----------+



In [ ]:
indexed.groupby('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[
    'Age',
    'Tonnage',
    'passengers',
    'length',
    'cabins',
    'cruise_cat'
],outputCol ='features' )
output = assembler.transform(indexed)

In [ ]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [ ]:
train_data , test_data = output.randomSplit([.8,.2])


In [ ]:
train_data.describe().show()

+-------+---------+-----------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|Ship_name|Cruise_line|               Age|          Tonnage|        passengers|           length|           cabins| passenger_density|              crew|        cruise_cat|
+-------+---------+-----------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|      131|        131|               131|              131|               131|              131|              131|               131|               131|               131|
|   mean| Infinity|       NULL|15.618320610687023|71.50374045801527|18.590458015267178|8.119160305343508|8.867099236641224|39.970916030534354| 7.823587786259548|4.8396946564885495|
| stddev|     NULL|       NULL|  7.81857552958355|36.93028327779621| 9.783977078924298|1.786169

In [ ]:
from pyspark.ml.regression import LinearRegression
regressor = LinearRegression(featuresCol='features',labelCol='crew')

In [ ]:
model = regressor.fit(train_data)

In [ ]:
result_train = model.evaluate(train_data)

In [ ]:
result_train.r2

0.9260600749335359

In [ ]:
y_pred = model.transform(test_data)

In [ ]:
result_test  = model.evaluate(test_data)

In [ ]:
result_test.r2

0.930369970631574

In [ ]:
y_pred.show()

+------------+----------------+---+------------------+----------+------+------+-----------------+-----+----------+--------------------+------------------+
|   Ship_name|     Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density| crew|cruise_cat|            features|        prediction|
+------------+----------------+---+------------------+----------+------+------+-----------------+-----+----------+--------------------+------------------+
|      Aurora|             P&O| 13|              76.0|     18.74|  8.86|  9.39|            40.55|  8.5|       8.0|[13.0,76.0,18.74,...| 8.830281083046334|
|     Diamond|        Princess|  9|             113.0|     26.74|  9.51| 13.37|            42.26|12.38|       2.0|[9.0,113.0,26.74,...|11.294945256501736|
|     Eurodam|Holland_American|  5|              86.0|     21.04|  9.36| 10.22|            40.87|  8.0|       3.0|[5.0,86.0,21.04,9...| 9.352139562747968|
|    Explorer| Royal_Caribbean| 13|             138.0|     31.14|  10.